# Tutorial: $k$NN for Information Theory
## Introduction
In this tutorial we show some basic usage of the UNITE toolbox to calculate entropy $H(X)$, mutual information $I(X; Y)$ and Kullback-Leibler (KL) divergence $D_{KL}(p||q)$ using k-nearest neighbours based methods.

## The Data
First we need to generate samples of a specific distribution from which there are known results for entropy, mutual information and KL divergence, or they can be calculated through numerical integration, for example. For this tutorial we we'll use the Gamma-Exponential distribution which is a 2D distribution which has the following probability density function (PDF):
$$
    p(x_1, x_2) = \frac{x_{1}^\theta e^{-x_{1} - x_{1} \cdot x_{2}}}{\Gamma\left ( \theta \right )}
$$
For this simple 2D case, the UNITE toolbox has auxiliary functions to generate samples through rejection sampling. We only need to define the PDF of the function we want to sample and the ranges in which to samples. So:

In [1]:
import numpy as np
from scipy.special import gamma

def pdf_gamma_exponential(x, y, theta): return (1 / gamma(theta)) * (x**theta) * np.exp(-x - x*y)
gexp_lims = [[0, 15], [0, 12]] # lower and upper limits for each dimension x1 and x2

Now we generate 10 000 samples of the Gamma-Exponential distribution and plot our samples to confirm. For the specific parameter of the distribution we will use $\theta=3$.

In [2]:
from unite_toolbox.utils import get_samples
samples1 = get_samples(func=pdf_gamma_exponential, limits=gexp_lims, n_samples=10_000, seed=0, params=3)

TypeError: pdf_gamma_exponential() got an unexpected keyword argument 'seed'